In [1]:
import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
model_paths = ["Save_File/All_building_seperate_05_31/1/checkpoints/1.ckpt",
                "Save_File/All_building_seperate_05_31/2/checkpoints/2.ckpt",
                "Save_File/All_building_seperate_05_31/3/checkpoints/3.ckpt",
                "Save_File/All_building_seperate_05_31/4/checkpoints/4.ckpt",
                "Save_File/All_building_seperate_05_31/5/checkpoints/5.ckpt"]

data_paths = ["Refined_Data/Seperated_Data/1.csv",
                "Refined_Data/Seperated_Data/2.csv",
                "Refined_Data/Seperated_Data/3.csv",
                "Refined_Data/Seperated_Data/4.csv",
                "Refined_Data/Seperated_Data/5.csv"]

input_length = 168
output_length = 24

In [3]:
for i in range(len(model_paths)):
    now_dataframe = pd.read_csv(data_paths[i], index_col=0)
    now_dataframe['num'] = now_dataframe['num'].apply(str)
    now_dataframe['Week'] = now_dataframe['Week'].apply(str)
    now_dataframe['24Hour'] = now_dataframe['24Hour'].apply(str)
    
    now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

    original = now_dataframe.iloc[0:input_length+output_length]["kWH"].tolist()
    prediction = now_dataframe.iloc[0:input_length+output_length]["kWH"].tolist()

    total_cycle = (len(now_dataframe) - input_length - output_length) // output_length

    for k in range(total_cycle):
        now_start_point = k*output_length
        now_input = now_dataframe.iloc[now_start_point:now_start_point+input_length].copy()

        now_input = TimeSeriesDataSet(
            now_input,
            time_idx="time_idx",
            target="kWH",
            group_ids=["num"],
            min_encoder_length=input_length//2,
            max_encoder_length=input_length,
            min_prediction_length=output_length,
            max_prediction_length=output_length,
            static_categoricals=["num"],
            static_reals=["non_electric_aircondition", "sunlight"],
            time_varying_known_categoricals=["Week", "24Hour"],
            time_varying_known_reals=["C", "m/s", "wet", "mm", "hr", "time_idx"],
            time_varying_unknown_categoricals=[],
            time_varying_unknown_reals=["kWH"],
            add_relative_time_idx=True,
            add_target_scales=True,
            add_encoder_length=True,
        )

        # get prediction, original data
        raw_predictions= now_best_tft.predict(now_input, mode="prediction").numpy().tolist()[0]
        originals = now_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

        prediction.extend(raw_predictions)
        original.extend(originals)

        # for next step, change dataframe's original value to predicted value
        for p in range(output_length):
            now_dataframe.iloc[now_start_point+input_length+p, now_dataframe.columns.get_loc("kWH")] = raw_predictions[p]

            
    plt.rcParams["figure.figsize"] = (30,10)
    fig = plt.figure()
    graph = fig.add_subplot(1, 1, 1)
    graph.plot(original, color='red')
    graph.plot(prediction, color='blue')
    plt.show()
    
    break
    